In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import os

In [1]:
import requests
from bs4 import BeautifulSoup
import time

# Base URL (to handle relative links)
base_url = "https://leginfo.legislature.ca.gov"  # Replace with your base URL
start_url = f"https://leginfo.legislature.ca.gov/faces/codes.xhtml"  # Replace with your starting page URL

# Storage for visited URLs and their scraped content
scraped_data = []

def fetch_page_content(url):
    """Fetch HTML content of a page."""
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.content
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

def scrape_and_follow_links(url, visited):
    """Scrape content and links from a page, then follow those links."""
    if url in visited:
        return  # Skip already visited URLs
    visited.add(url)
    
    html_content = fetch_page_content(url)
    if not html_content:
        return
    
    soup = BeautifulSoup(html_content, 'html.parser')
    div_content = soup.find("div", class_="tab_content")
    
    if div_content:
        # Extract text content
        text = div_content.get_text(strip=True)
        
        # Extract links and filter out JavaScript and anchor-only links
        links = [a['href'] for a in div_content.find_all('a', href=True)]
        full_links = [
            link if link.startswith("http") else f"{base_url}{link}" 
            for link in links 
            if not link.startswith("javascript:") and not link.startswith("#")
        ]
        
        # Save the current page's data
        scraped_data.append({"url": url, "text": text, "links": full_links})
        
        # Recursively scrape each link found
        for link in full_links:
            scrape_and_follow_links(link, visited)

# Start scraping
visited_urls = set()
scrape_and_follow_links(start_url, visited_urls)

In [3]:
with open("scraped_data.txt", "w", encoding="utf-8") as file:
    for page in scraped_data:
        file.write(f"URL: {page['url']}\n")
        file.write(f"Text Content:\n{page['text']}\n")
        file.write("Links Found:\n")
        for link in page['links']:
            file.write(f"- {link}\n")
        file.write("\n\n")

In [5]:
import json

with open("scraped_data.json", "w", encoding="utf-8") as file:
    json.dump(scraped_data, file, ensure_ascii=False, indent=4)

In [7]:
import csv

with open("scraped_data.csv", "w", newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["URL", "Text"])
    for page in scraped_data:
        writer.writerow([page['url'], page['text']])